In [1]:
import numpy as np
from sklearn.datasets import make_regression
import catboost
from joblib import load 
import pandas as pd
from os.path import join
import time 

In [78]:
def time_it(model_name, model, X,iters=10,onnx=False):
    res = []
    if not onnx:
        for i in range(iters):
            tic = time.time()
            pred = model.predict(X)
            toc = time.time()
            res.append((toc - tic))
            
    return pd.DataFrame({'model':model_name, 'time':res})


def time_it_onnx(sess,label_name, input_name, model_name,iter=10):
    res = []
    for i in range(iter):
        tic = time.time()
        predictions = sess.run([label_name],
                          {input_name:  X.astype(np.float32)})
        toc = time.time()
        res.append((toc - tic))
    return pd.DataFrame({'model':model_name, 'time':res})
    

In [88]:
models_path = 'models'


df_list = []


In [94]:
print (rt.get_device())
print (rt.get_available_providers())
#print (rt.get_all_providers())

CPU
['CPUExecutionProvider']


In [110]:
# Generación del dataset
n_samples = 1000000
n_features = 40
X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=0)

In [111]:
lr = load(join(models_path,'lr.joblib'))
cat_model =  load(join(models_path,'catb.joblib'))
rf_model = load(join(models_path,'rf.joblib'))

In [112]:
lr_results = time_it('lr', lr, X,iters=10)
df_list.append(lr_results)
lr_results

,model,time
0,lr,0.048117
1,lr,0.058975
2,lr,0.070092
3,lr,0.060488
4,lr,0.070090
5,lr,0.060393
6,lr,0.063102
7,lr,0.060382
8,lr,0.071976
9,lr,0.075512


In [113]:
cat_results = time_it('cat_model', cat_model, X,iters=10)
df_list.append(cat_results)

cat_results

,model,time
0,cat_model,11.796111
1,cat_model,11.727720
2,cat_model,11.315210
3,cat_model,11.435638
4,cat_model,11.327076
5,cat_model,12.117633
6,cat_model,12.273221
7,cat_model,12.454587
8,cat_model,12.165229
9,cat_model,12.551415


In [114]:
rf_results = time_it('rf_model', cat_model, X,iters=10)
df_list.append(rf_results)
rf_results

,model,time
0,rf_model,11.485145
1,rf_model,11.342507
2,rf_model,11.308372
3,rf_model,11.306891
4,rf_model,11.392119
5,rf_model,12.459421
6,rf_model,12.151239
7,rf_model,12.270667
8,rf_model,12.820041
9,rf_model,12.173771


# ONNX Inference

In [115]:
import onnxruntime as rt


In [116]:
sess = rt.InferenceSession(join(models_path,'catb.onnx'),providers=['CPUExecutionProvider'])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
model_name = 'cat_onnx'

cat_results_onnx = time_it_onnx(sess,label_name, input_name, model_name,iter=10)
df_list.append(cat_results_onnx)
cat_results_onnx



2023-02-15 02:40:58.695149097 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {1000000,1} for output predictions
2023-02-15 02:41:32.685213511 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {1000000,1} for output predictions
2023-02-15 02:42:07.894715995 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {1000000,1} for output predictions
2023-02-15 02:42:43.148982073 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {1000000,1} for output predictions
2023-02-15 02:43:17.067203014 [W:onnxruntime:, execution_frame.cc:835 VerifyOutputSizes] Expected shape from model of {-1} does not match actual shape of {1000000,1} for output predictions
2023-02-15 02:43:50.838779923 [W:onnxruntime:, executio

,model,time
0,cat_onnx,33.967247
1,cat_onnx,35.250432
2,cat_onnx,35.246189
3,cat_onnx,33.941690
4,cat_onnx,33.791053
5,cat_onnx,33.873540
6,cat_onnx,33.964868
7,cat_onnx,35.749935
8,cat_onnx,35.861356
9,cat_onnx,35.721595


In [117]:
sess = rt.InferenceSession(join(models_path,'rf.onnx'))

input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

model_name = 'rf_onnx'

rf_results_onnx = time_it_onnx(sess,label_name, input_name, model_name,iter=10)
df_list.append(rf_results_onnx)

rf_results_onnx

,model,time
0,rf_onnx,12.383910
1,rf_onnx,12.356936
2,rf_onnx,12.411336
3,rf_onnx,12.352478
4,rf_onnx,12.395233
5,rf_onnx,12.355812
6,rf_onnx,12.413200
7,rf_onnx,12.363297
8,rf_onnx,12.414368
9,rf_onnx,12.166269


In [118]:
sess = rt.InferenceSession(join(models_path,'lr.onnx'))


input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

model_name = 'lr_onnx'

lr_results = time_it_onnx(sess,label_name, input_name, model_name,iter=10)
df_list.append(lr_results)
lr_results

,model,time
0,lr_onnx,0.085064
1,lr_onnx,0.092187
2,lr_onnx,0.087553
3,lr_onnx,0.094150
4,lr_onnx,0.097218
5,lr_onnx,0.086588
6,lr_onnx,0.084421
7,lr_onnx,0.084126
8,lr_onnx,0.084252
9,lr_onnx,0.086486


In [122]:
metrics = pd.concat(df_list)

In [124]:
metrics.groupby('model').mean()


,time
model,
cat_model,4.783058
cat_onnx,19.075071
lr,0.024754
lr_onnx,0.046496
rf_model,4.718504
rf_onnx,6.794264


In [125]:
rt.get_available_providers()

['CPUExecutionProvider']

: 